In [70]:
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from webdriver_manager.chrome import ChromeDriverManager
from splinter import Browser
import pandas as pd

In [71]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'

browser.visit(url)

#retrieve the page with the requests module
response = browser.html

#create a BeautifulSoup object; parse with 'html parser'
soup = bs(response, 'html.parser')
news_title = soup.find_all('div', class_ = 'content_title')[1].text.strip()
news_p = soup.find_all('div', class_ = 'article_teaser_body')[0].text.strip()

In [72]:
#setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

#setup url
url1 = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [C:\Users\delac\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [73]:
# Visit URL using browser
browser.visit(url1)

# Use splinter to click into featured image to retrieve the image url
browser.links.find_by_partial_text('FULL IMAGE').click()

#set variable to this html to use soup and find end path for image
html = browser.html

# create soup object
soup_featured = bs(html, 'html.parser')

# retrieve the end_path and concatenate with the orginal url
end_path = soup_featured.find_all('img', class_='fancybox-image')[0]['src']
featured_image_url = url + end_path

In [74]:
# create URL for scraping
url2 = 'https://space-facts.com/mars/'

# create table
tables = pd.read_html(url2)[0]

tables.columns = ['description', 'mars']
table_html = tables.to_html(index = False)

In [75]:
url3 = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
# Visit URL using browser
browser.visit(url3)

# Use splinter to click into featured image to retrieve the image url
browser.find_by_css('a.product-item img').click()

#set variable to this html to use soup and find end path for image
html1 = browser.html

# create soup object
soup_images = bs(html1, 'html.parser')

# retrieve the end_path and concatenate with the orginal url
images = soup_images.find_all('div', class_ = 'downloads')[0].find('a')['href']

In [76]:
images

'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'